# Fastapi

`fastapi` is a python library that allows you to build APIs on top of python.

## Check it

- [Introductory and titorials on fastapi](https://fastapi.tiangolo.com/learn/).

## Run application

In the context of this site, my aim is to make each notebook runnable. To enhance convenience and prevent notebook cell stacking, I have opted to run different servers in Docker containers. This approach allows for seamless execution.

In this guide, I will explain how to build a container that will predominantly be utilized in the examples throughout the entire `fastapi` section. And `fastapi` basics steps the same time.

**Long story short** just use docker image with name `fastapi_experiment` just as it showen in section "run".

### Dockerfile

In the next cell is the docker file I am using for this example.

In [1]:
%%writefile fastapi/run_application_files/dockerfile
FROM python:3.11
COPY requrements.txt requrements.txt
RUN pip3 install -r requrements.txt
EXPOSE 8000

Overwriting fastapi/run_application_files/dockerfile


### `requrements.txt`

Python libraries you only needed to run the `fastapi` server. It is supposed to be used in the [dockerfile](#sec-dockerfile) described above.

In [2]:
%%writefile fastapi/run_application_files/requrements.txt
fastapi==0.103.1
uvicorn==0.23.2

Overwriting fastapi/run_application_files/requrements.txt


### Programme

You need to declare an object of class `fastapi.fastAPI`. Then use its decorators to add to your functions the ability to respond to certain requests.

So in the following example, I create `fastapi.fastAPI` under the name `my_first_app`, and create a function that will always respond `hello` to a `get` request.

In [3]:
%%writefile fastapi/run_application_files/get_started.py
from fastapi import FastAPI

my_first_app = FastAPI()

@my_first_app.get("/")
def say_hello():
    return "hello"

Overwriting fastapi/run_application_files/get_started.py


### Build the image

Image with name `fastapi_experiment`, created in the following cell, will be used in the other subsections of the fastapi section.

In [4]:
!docker build -t fastapi_experiment\
    ./fastapi/run_application_files/ &> /dev/null

### Run

To run the `fastapi` official documentation recomend to use `uvicorn` as web-server. So you need to use command:

`uvicon <path to python file with program>:<name of fastapi.fastAPI object in your program>.`

So in the following example, a docker container is run, tested and stopped:

- Created from the image named `fastapi_experiment` described in the docker file above;
- With the name `test_container` for container;
- The default port for `fastapi` is 8000, so port 8000 on the container is connected to port 8000 on the local machine;
- With volume that allows to read programme;
- And with the command created from the required pattern:
    - `--host 0.0.0.0` is used to make the application visible from the container, it's not necessary if you are using `uvicorn` without docker.

In [5]:
!docker run --rm -itd\
    --name test_container\
    -v ./fastapi/run_application_files/get_started.py:/get_started.py\
    -p 8000:8000 \
    fastapi_experiment \
    uvicorn --host 0.0.0.0 get_started:my_first_app >/dev/null

So now you can try it in your browser, but here I use the `curl` utility - it returns "hello", just as I declared in the programme.

In [6]:
!curl localhost:8000

curl: (56) Recv failure: Connection reset by peer


And we can also check what is happening inside the container. **Note** that the last line here is the log line for the http request from the previous cell.

In [7]:
!docker logs test_container

**Don't forget** to stop the image when you've finished playing with the container.

In [8]:
!docker stop test_container &> /dev/null

## Runtime update

The most convenient approach to experimenting with a container that contains fastapi is to dynamically swap the program, enabling the execution of multiple examples within a single container. For this purpose, I typically establish a connection between the utilized execution Python file and the container as a volume. This ensures that any changes made on the computer are promptly reflected within the container.

To implement this functionality, it is necessary to run uvicorn with the `--reload` flag. This flag enables uvicorn to monitor changes in the program and update accordingly.

Therefore, in the folowing cells:

- The container initiates with a program that responds with initial line.
- Testing it with curl yields the expected result of receiving initial line.
- Next, the reload.py file is modified to respond with updated line.
- Without delay, the same curl command is executed again, and we observe a response of updated line.

### Initial programme

In [11]:
%%writefile ./fastapi/run_application_files/reload.py
from fastapi import FastAPI

my_first_app = FastAPI()

@my_first_app.get("/")
def say_hello():
    return "initial line"

Overwriting ./fastapi/run_application_files/reload.py


In [12]:
!docker run --rm -itd\
    --name test_container\
    -v ./fastapi/run_application_files/reload.py:/reload.py\
    -p 8000:8000 \
    fastapi_experiment \
    uvicorn --host 0.0.0.0 --reload reload:my_first_app

831e85c13599a7e12a890a981b2b282dc89232253f072fcf7d32431105935043


In [13]:
!curl localhost:8000

curl: (56) Recv failure: Connection reset by peer


### Programme update

In [15]:
%%writefile ./fastapi/run_application_files/reload.py
from fastapi import FastAPI

my_first_app = FastAPI()

@my_first_app.get("/")
def say_hello():
    return "updated line"

Overwriting ./fastapi/run_application_files/reload.py


In [16]:
!curl localhost:8000

"updated line"

In [17]:
!docker stop test_container

test_container
